In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re

from matplotlib import pyplot as plt
from collections import Counter

# Analyzing NOSC of the S. cerevisiae proteome
Data from Xia et al. Nat Comms 2022. According to the "description of supplementary files" for that paper, the expression data are in units of molecules per cell. 

# Known issues
Expression data contains ≈50 proteins with multiple majority IDs. This is currently resolved by adding fictional IDs with data for the first of those IDs. Mostly these are small sequence variants with almost no effects on NOSC. 

In [2]:
samples_df = pd.read_excel('../data/proteomes/Scer/Xia_ScerCEN.PK.xlsx', sheet_name='samples', index_col=0)
raw_abund_df = pd.read_excel('../data/proteomes/Scer/Xia_ScerCEN.PK.xlsx', sheet_name='data')
nosc_df = pd.read_csv('../data/genomes/Scer/S288c/S288c_ref_prot_NOSC.csv')

raw_abund_df.head()

,majority_protein_ids,gene_name,S1,S2,S3,S4,S5,S6,S7,S8,...,S19,S20,S21,S22,S23,S24,S25,S26,S27,gene_function
0,P19097,FAS2,82452.270205,79779.523009,84618.632024,118520.675453,117488.905399,117159.998068,138977.483255,144138.512881,...,85063.315645,93305.693602,88758.854070,78471.524958,81948.073191,72495.589492,62546.099164,56921.725042,74664.084377,lipid metabolism
1,Q00955,ACC1,80980.039401,79181.536246,83903.335061,86035.271225,79691.734225,85331.089655,67574.941552,70916.743805,...,50902.456412,56970.872752,53373.639893,50524.880588,51603.976968,46927.974480,40988.634801,36394.374344,48429.078610,lipid metabolism
2,P07259,URA2,39354.814940,38889.618852,40467.620523,42649.741464,41779.577050,41948.514658,50886.047984,52430.940434,...,42549.345866,52366.807193,44114.959866,52879.038353,53874.218583,50731.992629,44877.261859,41946.010033,59242.826490,Nucleotides
3,P07149,FAS1,70992.082651,68074.744978,71888.271167,99290.383630,98498.151949,98328.464667,116102.009908,121262.573677,...,70537.897812,75996.292451,72702.279944,65449.876876,68052.050691,58567.930810,51500.342385,47162.486880,59724.931330,lipid metabolism
4,P06105,SCP160,15866.563114,15589.045692,16002.005154,17532.347516,16867.256547,16872.126352,22262.699513,23154.853437,...,25762.256045,28125.209247,25498.954660,30447.872840,30838.126270,30499.993751,28210.792000,23627.567774,35529.349996,ER


In [3]:
abund_ids = set(raw_abund_df.majority_protein_ids.values.tolist())
cds_ids = set(nosc_df.primary_accession.values.tolist())

# There are ≈50 entries where there was > 1 majority hit.
# That is: the relevant peptides report on multiple proteins, often
# because they are alternate translations of the same gene. 
missing_ids = abund_ids.difference(cds_ids)
print('Missing {0} ids'.format(len(missing_ids)))
shared_ids = abund_ids.intersection(cds_ids)

# The missing IDs are mostly due to isoforms of proteins that differ slightly in sequence. 
# This code identifies the individual IDs and makes a fictional row that represents the average
# of each of the isoforms.
print('Adding fictional IDs for those representing a mixture of isoforms')
lookup_table = nosc_df.set_index('primary_accession')
fakes = []
for my_id in missing_ids:
    NCs = []
    Ces = []
    for x in my_id.split(':'):
        if x in lookup_table.index:
            row = lookup_table.loc[x]
            NCs.append(row.NC)
            Ces.append(row.Ce)
            
    if len(NCs) == 0:
        continue
    print('Adding fictional protein for {0} representing {1} isoforms'.format(
        my_id, len(NCs)))
    
    NC = np.mean(NCs)
    Ce = np.mean(Ces)
    fake_protein = dict(primary_accession=my_id, NC=NC, Ce=Ce, NOSC=(Ce/NC))
    fakes.append(fake_protein)
    
extended_nosc_df = pd.concat([nosc_df, pd.DataFrame(fakes)], ignore_index=True)

# recheck which IDs are missing
cds_ids = set(extended_nosc_df.primary_accession.values.tolist())
missing_ids = abund_ids.difference(cds_ids)
print('After update, missing {0} IDs'.format(len(missing_ids)))
shared_ids = abund_ids.intersection(cds_ids)

Missing 45 ids
Adding fictional IDs for those representing a mixture of isoforms
Adding fictional protein for P0CX52:P0CX51 representing 2 isoforms
Adding fictional protein for P0CX34:P0CX33 representing 2 isoforms
Adding fictional protein for P0CX44:P0CX43 representing 2 isoforms
Adding fictional protein for Q3E754:P0C0V8 representing 2 isoforms
Adding fictional protein for P0CX83:P0CX82 representing 2 isoforms
Adding fictional protein for P0CX38:P0CX37 representing 2 isoforms
Adding fictional protein for P0CX48:P0CX47 representing 2 isoforms
Adding fictional protein for P15565-2:P15565 representing 2 isoforms
Adding fictional protein for P0CX26:P0CX25 representing 2 isoforms
Adding fictional protein for P38431:P38431-2 representing 2 isoforms
Adding fictional protein for P0CX85:P0CX84 representing 2 isoforms
Adding fictional protein for P07806-2:P07806 representing 2 isoforms
Adding fictional protein for P0CX50:P0CX49 representing 2 isoforms
Adding fictional protein for P07884-2:P078

In [4]:
# Checking the percentage of unmapped genes. 
data_cols = raw_abund_df.columns[2:-1]
shared_ids_list = list(shared_ids)
mapped_sum = raw_abund_df.set_index('majority_protein_ids')[data_cols].loc[shared_ids_list].sum()
total = raw_abund_df[data_cols].sum()
pct_diff = 100*(total - mapped_sum)/total

# Now that we've handled the isoforms, we're counting all the expression data
pct_diff.head()

TypeError: Passing a set as an indexer is not supported. Use a list instead.

In [ ]:
# Reshape the data to long-form
long_abund_df = raw_abund_df.drop('gene_function', axis=1).melt(
    id_vars=['majority_protein_ids', 'gene_name'], var_name='sample_name',
    value_name='copies_per_cell')

growth_rates = samples_df.loc[long_abund_df.sample_name].mu_per_hr
long_abund_df['growth_rate_hr'] = growth_rates.values

# use the extended_nosc_df to calculate the condition-dependent proteome NOSC
my_nosc_df = extended_nosc_df.set_index('primary_accession')
NCs = my_nosc_df.loc[long_abund_df.majority_protein_ids].NC.values
Ces = my_nosc_df.loc[long_abund_df.majority_protein_ids].Ce.values
NOSCs = my_nosc_df.loc[long_abund_df.majority_protein_ids].NOSC.values
long_abund_df['NC_per'] = NCs
long_abund_df['Ce_per'] = Ces
long_abund_df['NOSC'] = NOSCs
long_abund_df['NC_total'] = long_abund_df.copies_per_cell.multiply(NCs)
long_abund_df['Ce_total'] = long_abund_df.copies_per_cell.multiply(Ces)
long_abund_df['dataset'] = 'xia_2022'
long_abund_df['strain'] = 'CEN.PK113-7D'
long_abund_df['species'] = 'S. cerevisiae'
long_abund_df['organism_key'] = 'yeast'
long_abund_df['condition'] = 'chemostat_u' + samples_df.loc[long_abund_df.sample_name].mu_per_hr.astype(str).values
long_abund_df['fraction_transmembrane'] = my_nosc_df.loc[long_abund_df.majority_protein_ids].fraction_transmembrane.values
long_abund_df['fraction_transmembrane_C'] = my_nosc_df.loc[long_abund_df.majority_protein_ids].fraction_transmembrane_C.values

# Add annotation of the growth mode -- everything in this ref was done in chemostats
long_abund_df['growth_mode'] = 'chemostat'
# Add annotation of stress conds -- these are all glucose chemostat conds
long_abund_df['stress'] = False

# Save to CSV
long_abund_df.to_csv('../data/proteomes/Scer/Xia_protein_measurements.csv', index=False)

agg_dict = dict(NC_total='sum', Ce_total='sum', growth_rate_hr='first', growth_mode='first')
sample_noscs = long_abund_df.groupby(['sample_name']).agg(agg_dict)
sample_noscs['proteome_NOSC'] = sample_noscs.Ce_total / sample_noscs.NC_total

# reset growth rates -- didn't want to sum them
sample_noscs['growth_rate_hr'] = samples_df.loc[sample_noscs.index].mu_per_hr

sample_noscs.to_csv('../data/proteomes/Scer/Xia_proteome_NOSC_full.csv', index=False)
sample_noscs.head()


,NC_total,Ce_total,growth_rate_hr,growth_mode,proteome_NOSC
sample_name,,,,,
S1,1.244985e+11,-1.859931e+10,0.027,chemostat,-0.149394
S10,1.410082e+11,-2.146190e+10,0.152,chemostat,-0.152203
S11,1.529256e+11,-2.338499e+10,0.152,chemostat,-0.152917
S12,1.637752e+11,-2.509140e+10,0.152,chemostat,-0.153206
S13,8.950136e+10,-1.373303e+10,0.214,chemostat,-0.153439


In [ ]:
sample_noscs

,NC_total,Ce_total,growth_rate_hr,growth_mode,proteome_NOSC
sample_name,,,,,
S1,1.244985e+11,-1.859931e+10,0.027,chemostat,-0.149394
S10,1.410082e+11,-2.146190e+10,0.152,chemostat,-0.152203
S11,1.529256e+11,-2.338499e+10,0.152,chemostat,-0.152917
S12,1.637752e+11,-2.509140e+10,0.152,chemostat,-0.153206
S13,8.950136e+10,-1.373303e+10,0.214,chemostat,-0.153439
S14,1.254990e+11,-1.923848e+10,0.214,chemostat,-0.153296
S15,1.196692e+11,-1.846172e+10,0.214,chemostat,-0.154273
S16,1.240453e+11,-1.916775e+10,0.254,chemostat,-0.154522
S17,1.274194e+11,-1.966859e+10,0.254,chemostat,-0.154361


In [ ]:
# Mean of S. cer data since the replicates are reported separately 
agg_dict = dict(NC_total='mean', Ce_total='mean', proteome_NOSC='mean', sample_name='count')
sample_noscs_mean = sample_noscs.reset_index().groupby('growth_rate_hr').agg(agg_dict).rename(
    columns=dict(sample_name='sample_count'))
sample_noscs_mean['dataset'] = 'xia_2022'
sample_noscs_mean['strain'] = 'CEN.PK113-7D'
sample_noscs_mean['condition'] = 'chemostat_u' + sample_noscs_mean.index.astype(str)
sample_noscs_mean['growth_mode'] = 'chemostat'

sample_noscs_mean.to_csv('../data/proteomes/Scer/Xia_proteome_NOSC.csv', index=True)
sample_noscs_mean

,NC_total,Ce_total,proteome_NOSC,sample_count,dataset,strain,condition,growth_mode
growth_rate_hr,,,,,,,,
0.027,1.252822e+11,-1.866736e+10,-0.149003,3,xia_2022,CEN.PK113-7D,chemostat_u0.027,chemostat
0.044,1.294661e+11,-1.939256e+10,-0.149791,3,xia_2022,CEN.PK113-7D,chemostat_u0.044,chemostat
0.102,1.412387e+11,-2.139840e+10,-0.151504,3,xia_2022,CEN.PK113-7D,chemostat_u0.102,chemostat
0.152,1.525697e+11,-2.331276e+10,-0.152776,3,xia_2022,CEN.PK113-7D,chemostat_u0.152,chemostat
0.214,1.115565e+11,-1.714441e+10,-0.153669,3,xia_2022,CEN.PK113-7D,chemostat_u0.214,chemostat
0.254,1.271848e+11,-1.964471e+10,-0.154458,3,xia_2022,CEN.PK113-7D,chemostat_u0.254,chemostat
0.284,1.312617e+11,-2.032219e+10,-0.154824,3,xia_2022,CEN.PK113-7D,chemostat_u0.284,chemostat
0.334,1.322356e+11,-2.048149e+10,-0.154885,3,xia_2022,CEN.PK113-7D,chemostat_u0.334,chemostat
0.379,1.186361e+11,-1.839044e+10,-0.154985,3,xia_2022,CEN.PK113-7D,chemostat_u0.379,chemostat


In [ ]:
# amino acid counts per protein
aa_counts = [Counter(a) for a in nosc_df.aa_seq]
aa_counts_df = pd.DataFrame(aa_counts, index=nosc_df.primary_accession).replace({np.NaN: 0})
aa_counts_df.tail()

,M,T,G,F,K,V,S,Y,I,L,A,R,N,E,C,D,H,W,P,Q
primary_accession,,,,,,,,,,,,,,,,,,,,
P38088-2,12,30.0,41.0,33.0,60.0,51.0,39.0,18.0,37.0,56.0,49.0,36.0,23.0,61.0,6.0,49.0,13.0,7.0,30.0,16.0
P07263-2,11,27.0,40.0,20.0,52.0,33.0,40.0,17.0,36.0,42.0,52.0,20.0,15.0,40.0,5.0,35.0,6.0,4.0,15.0,16.0
P07806-2,15,58.0,59.0,38.0,108.0,68.0,63.0,34.0,74.0,88.0,71.0,38.0,44.0,92.0,11.0,69.0,22.0,24.0,51.0,31.0
P15565-2,12,30.0,32.0,17.0,53.0,35.0,41.0,23.0,30.0,44.0,37.0,24.0,42.0,38.0,11.0,23.0,12.0,5.0,30.0,15.0
P38784-2,4,11.0,9.0,3.0,19.0,8.0,13.0,10.0,9.0,19.0,4.0,6.0,9.0,17.0,1.0,11.0,1.0,1.0,4.0,17.0


In [ ]:
mask = samples_df.mu_per_hr == samples_df.mu_per_hr.max()
fast_samples = samples_df[mask].index
print(samples_df[mask])
mean_abund_fast_growth = raw_abund_df.set_index('majority_protein_ids')[fast_samples].mean(axis=1)

overlapping_ids = set(mean_abund_fast_growth.index).intersection(aa_counts_df.index)
tmp = aa_counts_df.loc[overlapping_ids].multiply(mean_abund_fast_growth.loc[overlapping_ids], axis=0).sum()
expression_weighted_aas = tmp / tmp.sum()
expression_weighted_aas.name = 'aa_freq'

expression_weighted_aas.to_csv('../data/proteomes/Scer/Xia_mu0.379_expression_weighted_aa_freqs.csv')

        mu_per_hr
sample           
S25         0.379
S26         0.379
S27         0.379
